<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Experimento 6 AutoGluon</h4>
</center>

In [1]:
# !pip install autogluon
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [2]:
df = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_ventas_product_v1.csv", header = None)
df.columns = ["timestamp", "item_id", "target"]
df.head(5)

,timestamp,item_id,target
0,2017-01-01,20001,934.77222
1,2017-01-01,20002,550.15707
2,2017-01-01,20003,1063.45835
3,2017-01-01,20004,555.91614
4,2017-01-01,20005,494.27011


In [3]:
df_product = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_products_v1.csv", header = None)
df_product.columns = ["item_id", "cat1", "cat2", "cat3", "brand", "sku_size"]
df_product.head(5)

,item_id,cat1,cat2,cat3,brand,sku_size
0,20280,HC,ROPA LAVADO,Liquido,LIMPIEX,900
1,20180,HC,ROPA LAVADO,Liquido,LIMPIEX,450
2,20332,HC,ROPA LAVADO,Liquido,LIMPIEX,120
3,20222,HC,ROPA LAVADO,Liquido,LIMPIEX,450
4,20288,HC,ROPA LAVADO,Liquido,LIMPIEX,900


In [6]:
df_related = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_related_product_v1.csv", header = None)
related_cols = ['timestamp',
 'item_id',
 'month',
 'quarter',
 'days_in_month',
 'sundays',
 'saturdays',
 'lag1m',
 'lag2m',
 'lag3m',
 'lag4m',
 'lag5m',
 'lag6m',
 'lag7m',
 'lag8m',
 'lag9m',
 'lag10m',
 'lag11m',
 'lag12m',
 'delta1',
 'delta2',
 'delta3',
 'delta4',
 'delta5',
 'delta6',
 'delta7',
 'delta8',
 'delta9',
 'delta10',
 'delta11',
 'delta12',
 'monthly_percentage',
 'lowerextremepoint',
 'upperextremepoint']
df_related.columns = related_cols
df_related.head(5)

,timestamp,item_id,month,quarter,days_in_month,sundays,saturdays,lag1m,lag2m,lag3m,...,delta6,delta7,delta8,delta9,delta10,delta11,delta12,monthly_percentage,lowerextremepoint,upperextremepoint
0,2017-01-01,20001,1,1,31,5,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.310483,1.0,0.0
1,2017-02-01,20001,2,1,28,4,4,934.77222,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.387267,1.0,0.0
2,2017-03-01,20001,3,1,31,4,4,798.01620,934.77222,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.798738,0.0,0.0
3,2017-04-01,20001,4,2,30,5,5,1303.35771,798.01620,934.77222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.223120,0.0,0.0
4,2017-05-01,20001,5,2,31,4,4,1069.96130,1303.35771,798.01620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.141096,0.0,0.0


In [7]:
df_extendido = pd.merge(df, df_related, how = "left", on = ["timestamp", "item_id"])
print(df_extendido.timestamp.max())
df_extendido.head(5)

2019-12-01


,timestamp,item_id,target,month,quarter,days_in_month,sundays,saturdays,lag1m,lag2m,...,delta6,delta7,delta8,delta9,delta10,delta11,delta12,monthly_percentage,lowerextremepoint,upperextremepoint
0,2017-01-01,20001,934.77222,1,1,31,5,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.310483,1.0,0.0
1,2017-01-01,20002,550.15707,1,1,31,5,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.990622,0.0,0.0
2,2017-01-01,20003,1063.45835,1,1,31,5,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.799958,0.0,0.0
3,2017-01-01,20004,555.91614,1,1,31,5,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.312602,0.0,0.0
4,2017-01-01,20005,494.27011,1,1,31,5,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.998823,0.0,0.0


In [8]:
data = TimeSeriesDataFrame.from_data_frame(df_extendido,
                                           id_column="item_id",
                                           timestamp_column="timestamp",
                                           static_features_df=df_product)
data.head()

,,target,month,quarter,days_in_month,sundays,saturdays,lag1m,lag2m,lag3m,lag4m,...,delta6,delta7,delta8,delta9,delta10,delta11,delta12,monthly_percentage,lowerextremepoint,upperextremepoint
item_id,timestamp,,,,,,,,,,,,,,,,,,,,,
20001,2017-01-01,934.77222,1,1,31,5,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.310483,1.0,0.0
20002,2017-01-01,550.15707,1,1,31,5,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.990622,0.0,0.0
20003,2017-01-01,1063.45835,1,1,31,5,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.799958,0.0,0.0
20004,2017-01-01,555.91614,1,1,31,5,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.312602,0.0,0.0
20005,2017-01-01,494.27011,1,1,31,5,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.998823,0.0,0.0


In [9]:
data.static_features.head()

,cat1,cat2,cat3,brand,sku_size
item_id,,,,,
20280,HC,ROPA LAVADO,Liquido,LIMPIEX,900
20180,HC,ROPA LAVADO,Liquido,LIMPIEX,450
20332,HC,ROPA LAVADO,Liquido,LIMPIEX,120
20222,HC,ROPA LAVADO,Liquido,LIMPIEX,450
20288,HC,ROPA LAVADO,Liquido,LIMPIEX,900


In [10]:
covariates = data.drop(columns = "target").columns

predictor = TimeSeriesPredictor(target='target',
                                prediction_length=2, 
                                freq="M",
                                eval_metric = "WQL",
                                known_covariates_names = covariates).fit(data, 
                                                                         num_val_windows=3,
                                                                         val_step_size=1)

Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels\ag-20240620_233441'


=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
GPU Count:          0
Memory Avail:       1.96 GB / 7.88 GB (24.8%)
Disk Space Avail:   5.61 GB / 110.21 GB (5.1%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': ['month',
                            'quarter',
                            'days_in_month',
                            'sundays',
                            'saturdays',
                            'lag1m',
                            'lag2m',
                            'lag3m',
                            'lag4m',
                            'lag5m',
                            'lag6m',
                            'lag7m',
   

In [11]:
predictor.leaderboard()

,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-0.052856,12.671370,5.567759,12
1,DirectTabular,-0.053677,10.336394,2212.036083,3
2,RecursiveTabular,-0.155657,2.334975,487.577084,2
3,AutoETS,-0.194559,37.847531,113.357014,7
4,PatchTST,-0.194902,0.652799,345.814017,11
5,DynamicOptimizedTheta,-0.201713,1.992769,58.167434,6
6,CrostonSBA,-0.209206,0.845729,32.353204,4
7,SeasonalNaive,-0.230813,0.898925,4.867838,1
8,AutoARIMA,-0.236277,93.094557,187.426256,8
9,NPTS,-0.280167,1.360631,2.797949,5


In [12]:
predictor.feature_importance()

Computing feature importance


,importance,stdev,n,p99_low,p99_high
cat1,0.000000,0.000000,5.0,0.000000,0.000000
cat2,0.000000,0.000000,5.0,0.000000,0.000000
cat3,0.000000,0.000000,5.0,0.000000,0.000000
brand,0.000000,0.000000,5.0,0.000000,0.000000
sku_size,-0.000112,0.000080,5.0,-0.000278,0.000053
month,0.001851,0.002916,5.0,-0.004153,0.007856
quarter,0.000774,0.001693,5.0,-0.002712,0.004260
days_in_month,0.000127,0.000505,5.0,-0.000912,0.001166
sundays,0.000166,0.000355,5.0,-0.000564,0.000896
saturdays,0.000351,0.001038,5.0,-0.001787,0.002489


In [13]:
known_covariates_future = df_related[df_related.timestamp > df.timestamp.max()]
# Llevamos las fechas futuras a fin de mes porque así lo espera el metodo predict
known_covariates_future['timestamp'] = pd.to_datetime(known_covariates_future['timestamp'])
known_covariates_future['timestamp'] = known_covariates_future['timestamp'] + pd.offsets.MonthEnd(0)
known_covariates_future['timestamp'] = known_covariates_future['timestamp'].dt.strftime('%Y-%m-%d')
print(known_covariates_future.timestamp.min(), known_covariates_future.timestamp.max())
known_covariates_future = TimeSeriesDataFrame.from_data_frame(known_covariates_future,
                                                              id_column="item_id",
                                                              timestamp_column="timestamp")
known_covariates_future.head()

2020-01-31 2020-02-29


month  quarter  days_in_month  sundays  saturdays  \
item_id timestamp                                                       
20001   2020-01-31      1        1             31        4          4   
        2020-02-29      2        1             29        4          5   
20002   2020-01-31      1        1             31        4          4   
        2020-02-29      2        1             29        4          5   
20003   2020-01-31      1        1             31        4          4   

                         lag1m       lag2m       lag3m       lag4m  \
item_id timestamp                                                    
20001   2020-01-31  1504.68856  1397.37231  1561.50552  1660.00561   
        2020-02-29         NaN  1504.68856  1397.37231  1561.50552   
20002   2020-01-31  1087.30855  1423.57739  1979.53635  1090.18771   
        2020-02-29         NaN  1087.30855  1423.57739  1979.53635   
20003   2020-01-31   892.50129   948.29393  1081.36645   967.77116   

                         lag5m  ...  delta6  delta7  delta8  delta9  delta10  \
item_id timestamp               ...                                            
20001   2020-01-31  1261.34529  ...     NaN     NaN     NaN     NaN      NaN   
        2020-02-29  1660.00561  ...     NaN     NaN     NaN     NaN      NaN   
20002   2020-01-31   813.78215  ...     NaN     NaN     NaN     NaN      NaN   
        2020-02-29  1090.18771  ...     NaN     NaN     NaN     NaN      NaN   
20003   2020-01-31   635.59563  ...     NaN     NaN     NaN     NaN      NaN   

                    delta11  delta12  monthly_percentage  lowerextremepoint  \
item_id timestamp                                                             
20001   2020-01-31      NaN      NaN            7.308178                NaN   
        2020-02-29      NaN      NaN            7.212629                NaN   
20002   2020-01-31      NaN      NaN            8.980967                NaN   
        2020-02-29      NaN      NaN            7.394508                NaN   
20003   2020-01-31      NaN      NaN           10.242019                NaN   

                    upperextremepoint  
item_id timestamp                      
20001   2020-01-31                NaN  
        2020-02-29                NaN  
20002   2020-01-31                NaN  
        2020-02-29                NaN  
20003   2020-01-31                NaN  

[5 rows x 32 columns]

In [14]:
predictions = predictor.predict(data,
                                known_covariates = known_covariates_future)

data with frequency 'None' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [15]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1401.573364
3,20002,1086.291748
5,20003,810.216919
7,20004,583.750427
9,20005,546.370300


In [16]:
predictions_v1.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_autogluon_static_features_and_covariates_newFeatures.csv", sep = ",", index = False)